# Installing Libraries

In [ ]:
!pip install -q numpy pandas altair scikit-learn hyperopt sklearn-genetic-opt optunity optuna hpbandster

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 5.4 MB/s eta 0:00:00


이번 실습에서는 여러 개의 라이브러리가 필요하다. 설치할 라이브러리는 다음과 같다:
* [hyperopt](https://hyperopt.github.io/hyperopt/): Simulated Annealing을 활용한 초매개변수 최적화
* [sklearn-genetic-opt](https://sklearn-genetic-opt.readthedocs.io/en/stable/index.html): Genetic algorithm을 활용한 초매개변수 최적화
* [optunity](https://optunity.readthedocs.io/en/latest/index.html): Particle Swarm Optimization을 활용한 초매개변수 최적화
* [optuna](https://optuna.readthedocs.io/en/stable/index.html): Successive Halving, Hyperband를 활용한 초매개변수 최적화
* [HpBandSter](https://automl.github.io/HpBandSter/build/html/index.html): Bayesian Optimization and Hyperband를 활용한 초매개변수 최적화

# Data Preparation

데이터로는 지난 시간에도 사용했던 은행의 전화 마케팅 성공 여부를 예측하는 [Banking Dataset - Marketing Targets](https://www.kaggle.com/datasets/prakharrathi25/banking-dataset-marketing-targets)을 이용한다.

지난 시간에 했던 대로 범주형 데이터는 One-hot Encoding을 통해 숫자로 바꿔주겠다.

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder


URL_DATA = 'https://drive.usercontent.google.com/download?export=download&confirm=t&id=1_XXEOnEr8hUqNhFDu8x4Z7Rvb-4ZUTcm'
DATA = pd.read_csv(URL_DATA, sep=';')
X, y = DATA.drop('y', axis=1), DATA['y']
X_NUM, X_CAT = X.select_dtypes(include='number'), X.select_dtypes(exclude='number')

encoder = OneHotEncoder(drop='if_binary', sparse_output=False)

X_CAT = pd.DataFrame(
    encoder.fit_transform(X_CAT),
    columns=encoder.get_feature_names_out()
)

X = pd.concat([X_NUM, X_CAT], axis=1)
y = y.replace({'no': 0, 'yes': 1})

이제, 훈련 데이터셋과 검증 데이터셋으로 분할하자.

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit


splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.4, random_state=42)
I_train, I_test = next(splitter.split(X, y))
X_train, y_train = X.iloc[I_train].values, y.iloc[I_train].values
X_test, y_test = X.iloc[I_test].values, y.iloc[I_test].values

# Default Performance

먼저 기본 성능부터 다시 확인해보자.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score


model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
f1_score(y_test, y_pred, pos_label=1)

0.4908759124087591

# Heuristic Search

먼저, 이론적인 바탕보다는 시행 착오를 거쳐서 초매개변수를 탐색하는 방법인 Heuristic Search를 해보자.

## Simulated Annealing

Simulated Annealing은 지난 시간에 사용해봤던 [hyperopt](https://github.com/hyperopt/hyperopt)에 구현되어 있다. 물론, hyperopt를 사용하는 것이니만큼, 목적 함수와 검색 공간을 정의해야 한다.

In [ ]:
from hyperopt import hp
import numpy as np
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold


search_space = {
    'n_estimators': 5 + hp.randint('n_estimators', 195), # 5 ~ 200 사이의 정수를 균일 분포 확률Uniformly Random로 뽑는다.
    'criterion': hp.choice('criterion', ['gini', 'entropy']), # 'gini' 와 'entropy'를 중 하나를 균일 분포 확률Uniformly Random로 뽑는다.
    'max_depth': 3 + hp.randint('max_depth', 17), # 3 ~ 20 사이의 정수를 균일 분포 확률Uniformly Random로 뽑는다.
    'class_weight': hp.choice('class_weight', ['balanced', 'balanced_subsample']), # 'balanced' 와 'balanced_subsample'를 중 하나를 균일 분포 확률Uniformly Random로 뽑는다.
    'min_samples_split': hp.uniform('min_samples_split', 0.01, 0.5), # 0.01 ~ 0.5 사이의 실수를 균일 분포 확률Uniformly Random로 뽑는다.
}

def objective(hyperparams):
    splitter = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = []

    for I_train, I_test in splitter.split(X_train, y_train):
        X_inner_train, X_inner_test, y_inner_train, y_inner_test = X_train[I_train], X_train[I_test], y_train[I_train], y_train[I_test]
        model = RandomForestClassifier(random_state=42)
        model.set_params(**hyperparams)
        model.fit(X_inner_train, y_inner_train)
        y_pred = model.predict(X_inner_test)
        f1 = f1_score(y_true=y_inner_test, y_pred=y_pred, pos_label=1)
        scores.append(f1)

    return -np.mean(scores)

다음은 최적화 실행이다. 아래의 코드를 확인하자.

In [ ]:
from hyperopt import fmin, anneal


best = fmin(
    fn=objective, # 정의한 목적 함수를 입력한다.
    space=search_space, # 검색 공간을 정의한다.
    algo=anneal.suggest, # Simulated Annealing을 활용한다.
    max_evals=50 # 총 50회의 초매개변수 조합을 탐색한다.
)

100%|██████████| 50/50 [03:51<00:00,  4.63s/trial, best loss: -0.5384130731151785]


찾아낸 최적의 초매개변수는 다음과 같다.

In [ ]:
best

{'class_weight': 0,
 'criterion': 0,
 'max_depth': 10,
 'min_samples_split': 0.014733638670721999,
 'n_estimators': 43}

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score


model = RandomForestClassifier(
    random_state=42,
    class_weight='balanced',
    criterion='gini',
    max_depth=10,
    min_samples_split=0.014733638670721999,
    n_estimators=43
).fit(X_train, y_train)

y_pred = model.predict(X_test)
f1_score(y_test, y_pred, pos_label=1)

0.5285268237297758

## Genetic Algorithm

이번엔 여러 개체로 구성된 모집단이 서로 자연 선택, 교배, 돌연변이를 일으키면서 최적의 매개변수를 찾는 방법인 Genetic Algorithm을 실습해보자. Python 기반의 Genetic Algorithm 구현체는 [DEAP](https://deap.readthedocs.io/en/master/)가 가장 유명하다. 하지만, DEAP는 초매개변수 최적화 용도로 개발된 것은 아니다. 따라서, DEAP에 초매개변수 최적화를 적용하려면 별도의 구현이 좀 필요하다.

물론, 다행히도 DEAP와 scikit-learn을 결합해서 초매개변수 최적화 용도로 만들어 둔 라이브러리인 [sklearn-genetic-opt](https://sklearn-genetic-opt.readthedocs.io/en/stable/index.html)가 있다.

먼저, 초매개변수의 검색 공간을 정의하자.

In [ ]:
from sklearn_genetic.space import Categorical, Integer, Continuous


search_space = {
    'n_estimators': Integer(5, 200), # 5 ~ 200 사이의 정수를 균일 분포 확률Uniformly Random로 뽑는다.
    'criterion': Categorical(['gini', 'entropy']), # 'gini' 와 'entropy'를 중 하나를 균일 분포 확률Uniformly Random로 뽑는다.
    'max_depth': Integer(3, 20), # 3 ~ 20 사이의 정수를 균일 분포 확률Uniformly Random로 뽑는다.
    'class_weight': Categorical(['balanced', 'balanced_subsample']), # 'balanced' 와 'balanced_subsample'를 중 하나를 균일 분포 확률Uniformly Random로 뽑는다.
    'min_samples_split': Continuous(0.01, 0.5), # 0.01 ~ 0.5 사이의 실수를 균일 분포 확률Uniformly Random로 뽑는다.
}

그 다음에는 [sklearn.model_selection.GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV)나 [sklearn.model_selection.RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV)와 유사하게 [sklearn_genetic.GASearchCV](https://sklearn-genetic-opt.readthedocs.io/en/stable/api/gasearchcv.html)를 쓰면 된다.


In [ ]:
from sklearn_genetic import GASearchCV
from sklearn.ensemble import RandomForestClassifier


optimizer = GASearchCV(
    estimator=RandomForestClassifier(random_state=42), # 초매개변수를 찾을 학습 모델,
    param_grid=search_space, # 초매개변수 검색 공간
    scoring='f1', # 성능 측정치; F1 score를 활용한다.
    criteria='max', # 성능 측정치를 최대화하는 방향으로 최적화를 수행한다.
    cv=3, # 내부 CV의 설정; 3를 넣을 시 Stratified 3-Fold CV를 수행한다. 또는, Cross-Validation 시간에 배웠던 Splitter 클래스들을 넣어도 된다.
    population_size=20, # 모집단의 개수를 의미한다.
    generations=10, # 몇 세대나 생성할지를 의미한다
    tournament_size=3, # 선택 시 몇 개체로 토너먼트를 할 지를 의미한다.
    crossover_probability=0.2, # 교배 시 각 개체가 가지고 있는 초매개변수가 교환될 확률을 의미한다.
    mutation_probability=0.1, # 자식 개체의 초매개변수가 변이할 확률을 의미한다.
)

optimizer.fit(X_train, y_train)

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	20    	0.453604	0.0290435  	0.519186   	0.420805   
1  	9     	0.475616	0.0266384  	0.519186   	0.431452   
2  	11    	0.499506	0.0219322  	0.519186   	0.455506   
3  	16    	0.513135	0.00449957 	0.519186   	0.505694   
4  	14    	0.516378	0.00156661 	0.519186   	0.515253   
5  	12    	0.517791	0.0033353  	0.529294   	0.515253   
6  	10    	0.521153	0.0044334  	0.532136   	0.516317   
7  	14    	0.523273	0.00429845 	0.532136   	0.518566   
8  	13    	0.529142	0.00292844 	0.532136   	0.523468   
9  	12    	0.53119 	0.00125388 	0.532136   	0.529294   
10 	10    	0.531947	0.000624312	0.532136   	0.529294   


GASearchCV(estimator=RandomForestClassifier(class_weight='balanced_subsample',
                                            criterion='entropy', max_depth=20,
                                            min_samples_split=0.026357045474641412,
                                            n_estimators=94, random_state=42),
           generations=10, mutation_probability=0.1,
           param_grid={'class_weight': <sklearn_genetic.space.space.Categorical object at 0x79ef126825c0>,
                       'criterion': <sklearn_genetic.space.space.Categorical object at 0x79ef12681f30>,
                       'max_depth': <sklearn_genetic.space.space.Integer object at 0x79ef12683100>,
                       'min_samples_split': <sklearn_genetic.space.space.Continuous object at 0x79ef126822c0>,
                       'n_estimators': <sklearn_genetic.space.space.Integer object at 0x79ef12683640>},
           population_size=20, return_train_score=True, scoring='f1')

찾아낸 최적의 초매개변수와 검증 데이터셋에 대한 성능은 다음과 같다.

In [ ]:
from sklearn.metrics import f1_score


y_pred = optimizer.best_estimator_.predict(X_test)

print(f'Best F1 Score: {optimizer.best_score_}')
print('Best Hyperparameters:')
print(optimizer.best_params_)
print(f'F1 Score on validation set: {f1_score(y_test, y_pred, pos_label=1)}')

Best F1 Score: 0.5321357087739744
Best Hyperparameters:
{'n_estimators': 94, 'criterion': 'entropy', 'max_depth': 20, 'class_weight': 'balanced_subsample', 'min_samples_split': 0.026357045474641412}
F1 Score on validation set: 0.5286073123081217


## Particle Swarm Optimization

Heuristic Search의 마지막으로, 무리 지성을 활용하는 Particle Swarm Optimization을 사용해보자. 다행히 PSO를 직접 구현할 필요 없이 [Optunity](https://optunity.readthedocs.io/en/latest/index.html)를 활용하면 된다. 하지만, 문제는 Optunity의 경우 오직 연속형 실수에 대해서만 초매개변수 탐색을 하도록 구현되어 있다는 것이다. 따라서, 목적 함수에 좀 수정이 필요하다. 먼저, 목적 함수부터 구현해보자.

In [ ]:
import numpy as np
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold


def objective(n_estimators, criterion, max_depth, class_weight, min_samples_split):
    n_estimators = int(n_estimators) # 실수를 강제로 정수형으로 바꾼다.
    criterion = 'gini' if criterion < 0.5 else 'entropy' # 실수를 강제로 범주형 초매개변수로 바꾼다.
    max_depth = int(max_depth) # 실수를 강제로 정수형으로 바꾼다.
    class_weight = 'balanced' if class_weight < 0.5 else 'balanced_subsample' # 실수를 강제로 범주형 초매개변수로 바꾼다.

    splitter = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = []

    for I_train, I_test in splitter.split(X_train, y_train):
        X_inner_train, X_inner_test, y_inner_train, y_inner_test = X_train[I_train], X_train[I_test], y_train[I_train], y_train[I_test]
        model = RandomForestClassifier(
            random_state=42,
            n_estimators=n_estimators,
            criterion=criterion,
            max_depth=max_depth,
            class_weight=class_weight,
            min_samples_split=min_samples_split
        )
        model.fit(X_inner_train, y_inner_train)
        y_pred = model.predict(X_inner_test)
        f1 = f1_score(y_true=y_inner_test, y_pred=y_pred, pos_label=1)
        scores.append(f1)

    return np.mean(scores)

위에서처럼 실수형 초매개변수를 강제로 정수형 초매개변수 또는 범주형 초매개변수로 바꿔주었다.

다음엔 초매개변수 검색 공간을 정의하자.

In [ ]:
search_space = {
    'n_estimators': [5, 200], # 5 ~ 200 사이의 실수를 균일 분포 확률Uniformly Random로 뽑는다.
    'criterion': [0, 1], # 0 ~ 1 사이의 실수를 균일 분포 확률Uniformly Random로 뽑는다.
    'max_depth': [3, 20], # 3 ~ 20 사이의 실수를 균일 분포 확률Uniformly Random로 뽑는다.
    'class_weight': [0, 1], # 0 ~ 1 사이의 실수를 균일 분포 확률Uniformly Random로 뽑는다.
    'min_samples_split': [0.01, 0.5] # 0.01 ~ 0.5 사이의 실수를 균일 분포 확률Uniformly Random로 뽑는다.
}

마지막으로, PSO를 통해 초매개변수 최적화를 해보자.

In [ ]:
import optunity


best_params, logs, space = optunity.maximize(
    f=objective,
    num_evals=50, # PSO에서 위치 업데이트를 할 횟수
    solver_name='particle swarm', # Particle Swarm Optmiziation을 사용
    **search_space
)

In [ ]:
best_params

{'n_estimators': 123.9717779124975,
 'criterion': 0.8545833634311287,
 'max_depth': 10.305683593750004,
 'class_weight': 0.5657322328390989,
 'min_samples_split': 0.04352480468750003}

PSO를 통해 찾아낸 최적의 초매개변수를 사용해서, 검증 데이터셋에 대한 성능을 확인해보자.

In [ ]:
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier


model = RandomForestClassifier(
    random_state=42,
    class_weight='balanced_subsample',
    criterion='entropy',
    max_depth=int(10.305683593750004),
    min_samples_split=0.04352480468750003,
    n_estimators=int(123.9717779124975)
).fit(X_train, y_train)

y_pred = model.predict(X_test)
f1_score(y_test, y_pred, pos_label=1)

0.5205252246026261

# Multi-Fidelity Optimization
이번엔 학습 모델의 Fidelity를 조정해서, 낮은 Fidelity Model에서는 많은 수의 초매개변수 후보를 탐색하고, 높은 Fidelity Model에서는 적은 수의 (검증된) 초매개변수 후보를 탐색하는 Multi-Fidelity Optimization을 해보자.


## Successive Halving

가장 먼저 해볼 건 Succesive Halving이다. 수업 시간에 배운 다양한 MFO 기법의 기반이 되는 방법이니 만큼 여러 구현체가 존재한다. 게다가, 최근에는 scikit-learn에서도 Successive Halving 구현체를 제공한다! [sklearn.model_selection.GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV)와 [sklearn.model_selection.RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV)의 Successive Halving 버전인 [sklearn.model_selection.HalvingGridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html#sklearn.model_selection.HalvingGridSearchCV)와 [sklearn.model_selection.HalvingRandomSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html#sklearn.model_selection.HalvingRandomSearchCV)가 바로 그것이다.

지난 시간에 했던 GridSearchCV 또는 RandomizedSearchCV와 거의 유사하다. 먼저 초매개변수 검색 공간을 정의하자.

In [ ]:
import scipy.stats as st


search_space = {
    'n_estimators': st.randint(5, 200), # 5 ~ 100 사이의 정수를 균일 분포 확률Uniformly Random로 뽑는다.
    'criterion': ['gini', 'entropy'], # 'gini' 와 'entropy'를 중 하나를 균일 분포 확률Uniformly Random로 뽑는다.
    'max_depth': st.randint(3, 20), # 3 ~ 20 사이의 정수를 균일 분포 확률Uniformly Random로 뽑는다.
    'class_weight': ['balanced', 'balanced_subsample'], # 'balanced' 와 'balanced_subsample'를 중 하나를 균일 분포 확률Uniformly Random로 뽑는다.
    'min_samples_split': st.truncnorm(a=0, b=0.5, loc=0.005, scale=0.01), # 평균은 0.005, 표준 편차는 0.01이, 최소값 0, 최대값 0.5인 정규 분포Normal Distribution에서 뽑는다.
}

그럼 남은 건, HalvingRandomSearchCV를 RandomizedSearchCV와 동일하게 사용하는 것 뿐이다.

In [ ]:
from sklearn.experimental import enable_halving_search_cv # 아직 Success Halving는 실험적 기능이라, 이를 먼저 실행해야 한다.
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.ensemble import RandomForestClassifier


optimizer = HalvingRandomSearchCV(
    estimator=RandomForestClassifier(random_state=42), # 초매개변수를 찾을 학습 모델,
    param_distributions=search_space, # 초매개변수 검색 공간,
    scoring='f1', # 성능 측정치; F1 score를 활용한다,
    cv=3, # 내부 CV의 설정; 3를 넣을 시 Stratified 3-Fold CV를 수행한다. 또는, Cross-Validation 시간에 배웠던 Splitter 클래스들을 넣어도 된다.
    n_jobs=-1, # 병렬로 실행할 작업의 개수를 의미한다. -1일 경우 모든 프로세서를 활용한다는 의미다.
    resource='n_samples', # 데이터의 수를 자원으로 사용한다.
    n_candidates=100, # 초기에 100개의 초매개변수 후보로 시작한다
    min_resources=20, # 초기에 20개의 데이터로 시작한다.
    factor=3, # 매 Halving 단계에서 감소시킬/증가시킬 초개매변수 후보/자원의 양을 의미한다; 3일 경우, 초매개변수 후보들은 1/3으로 줄이고, 자원의 양은 3배로 늘린다.
)

optimizer.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


HalvingRandomSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42),
                      min_resources=20, n_candidates=100, n_jobs=-1,
                      param_distributions={'class_weight': ['balanced',
                                                            'balanced_subsample'],
                                           'criterion': ['gini', 'entropy'],
                                           'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79eea7e67340>,
                                           'min_samples_split': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x79eea7e67370>,
                                           'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79ef134264d0>},
                      scoring='f1')

찾아낸 최적의 초매개변수와 성능치는 다음과 같다:

In [ ]:
from sklearn.metrics import f1_score


print(f'Best F1 Score: {optimizer.best_score_}')
print('Best Hyperparameters:')
print(optimizer.best_params_)
y_pred = optimizer.best_estimator_.predict(X_test)
print(f'F1 Score on validation set: {f1_score(y_test, y_pred, pos_label=1)}')

Best F1 Score: 0.47596104228369224
Best Hyperparameters:
{'class_weight': 'balanced_subsample', 'criterion': 'entropy', 'max_depth': 5, 'min_samples_split': 0.005339500060267382, 'n_estimators': 108}
F1 Score on validation set: 0.508728899148752


Succesive Halving는 optuna에서도 제공한다. 사실, 이 이후에 할 Hyperband나 Bayesian Optimization and Hyperband를 하려면 optuna를 사용하는 게 더 좋다.

지난 시간에 했던 optuna 관련 내용 중 초매개변수를 추출하기 위한 공간을 정의하는 Sampler 인스턴스를 정의했던 것을 기억할 것이다. 이 외에도, optuna는 추출한 초매개변수 후보들을 줄이는 Pruner 인스턴스도 존재한다. 직관적으로, Successive Halving이나 Hyperband는 이미 초기에 추출했던 초매개변수 후보들을 **줄이는** Pruner의 일종임을 알 수 있을 것이다.

optuna에서 Pruner가 동작하기 위해서는 다음과 같이 목적함수를 정의해야 한다.

In [ ]:
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
import optuna
import numpy as np


def objective(trial: optuna.Trial):
    n_estimators = trial.suggest_int('n_estimators', 5, 200)
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    max_depth = trial.suggest_int('max_depth', 3, 20)
    class_weight = trial.suggest_categorical('class_weight', ['balanced', 'balanced_subsample'])
    min_samples_split = trial.suggest_float('min_samples_split', 0.01, 0.05)

    # 여기서부터 Successive Halving에서 고려하는 자원양에 대한 학습 정도를 구현해야 한다.
    # scikit-learn은 알아서 자원양(데이터 수)를 증가시켜주었지만, optuna에서는 목적 함수내에서 구현할 필요가 있다.
    # 물론, 자유도 측면에서는 좋다.

    min_samples = 100 # 초기에 시작할 최소 샘플수
    factor = 3 # 매 반복마다 늘릴 샘플 수
    step = 0 # 반복 횟수

    while True:
        n_samples = min_samples * (factor ** step)
        is_completed = False
        # 매 반복마다 훈련에 사용할 데이터 양을 3배씩 늘려주자.
        if n_samples > len(X_train):
            X_resource, y_resource = X_train, y_train
            is_completed = True
        else:
            splitter = StratifiedShuffleSplit(n_splits=1, train_size=n_samples, random_state=42)
            I_resource, _ = next(splitter.split(X_train, y_train))
            X_resource, y_resource = X_train[I_resource], y_train[I_resource]

        # 이제 자원으로 사용할 데이터를 활용해 3-Fold CV를 하자
        splitter = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
        scores = []
        for I_train, I_test in splitter.split(X_resource, y_resource):
            X_inner_train, X_inner_test, y_inner_train, y_inner_test = X_resource[I_train], X_resource[I_test], y_resource[I_train], y_resource[I_test]
            model = RandomForestClassifier(
                random_state=42,
                n_estimators=n_estimators,
                criterion=criterion,
                max_depth=max_depth,
                class_weight=class_weight,
                min_samples_split=min_samples_split
            )
            model.fit(X_inner_train, y_inner_train)
            y_pred = model.predict(X_inner_test)
            f1 = f1_score(y_true=y_inner_test, y_pred=y_pred, pos_label=1)
            scores.append(f1)

        # 만약 모든 자원을 사용한 것이라면 Trial을 종료한다
        if is_completed:
            return np.mean(scores)
        # 그렇지 않다면, 중간 성능치 보고한다.
        # optuna는 여기서 보고한 성능치를 바탕으로,
        # 이 초매개변수의 제거 여부를 판단한다.
        else:
            trial.report(
                value=np.mean(scores),
                step=step
            )
        # 이 부분을 꼭 넣어줘야 Pruner가 이 Trial을 제거할 수 있다.
        if trial.should_prune():
            raise optuna.TrialPruned()
        step = step + 1

그 다음엔, 아래처럼 [optuna.pruners.SuccessiveHalvingPruner](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.pruners.SuccessiveHalvingPruner.html)를 pruner로 활용하면 된다,

In [ ]:
import optuna
from optuna.samplers import RandomSampler
from optuna.pruners import SuccessiveHalvingPruner


study = optuna.create_study(
    sampler=RandomSampler(), # Successive Halving은 기본적으로 초기 초매개변수를 무작위로 찾기 때문에, RandomSampler를 사용한다.
    pruner=SuccessiveHalvingPruner(reduction_factor=3), # Successive Halving을 통해 초매개변수 후보들을 줄여나간다.
    direction='maximize', # F1 Score를 최대화하는 초매개변수를 찾는다
)

optuna.logging.set_verbosity(optuna.logging.WARNING) # 불필요한 로그는 보여주지 말자.

study.optimize(
    func=objective, # 목적 함수를 의미한다
    n_trials=50, # 탐색할 초매개변수들의 개수를 의미한다
    show_progress_bar=True # 표준 출력에 프로그레스 바를 보여준다.
)

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
study.best_params

{'n_estimators': 93,
 'criterion': 'entropy',
 'max_depth': 16,
 'class_weight': 'balanced',
 'min_samples_split': 0.013542441252543495}

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score


model = RandomForestClassifier(
    random_state=42,
    **study.best_params
).fit(X_train, y_train)

y_pred = model.predict(X_test)
f1_score(y_test, y_pred, pos_label=1)

0.5438494934876991

## Hyperband

Hyperband는 간단하다. optuna에서 Pruner를 [optuna.pruners.HyperbandPruner](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.pruners.HyperbandPruner.html)로 바꿔주기만 하면 된다.

In [ ]:
import optuna
from optuna.samplers import RandomSampler
from optuna.pruners import HyperbandPruner


study = optuna.create_study(
    sampler=RandomSampler(), # Successive Halving은 기본적으로 초기 초매개변수를 무작위로 찾기 때문에, RandomSampler를 사용한다.
    pruner=HyperbandPruner(reduction_factor=3), # Successive Halving을 통해 초매개변수 후보들을 줄여나간다.
    direction='maximize', # F1 Score를 최대화하는 초매개변수를 찾는다
)

optuna.logging.set_verbosity(optuna.logging.WARNING) # 불필요한 로그는 보여주지 말자.

study.optimize(
    func=objective, # 목적 함수를 의미한다
    n_trials=50, # 탐색할 초매개변수들의 개수를 의미한다
    show_progress_bar=True # 표준 출력에 프로그레스 바를 보여준다.
)

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
study.best_params

{'n_estimators': 60,
 'criterion': 'gini',
 'max_depth': 15,
 'class_weight': 'balanced_subsample',
 'min_samples_split': 0.01042357202241754}

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score


model = RandomForestClassifier(
    random_state=42,
    **study.best_params
).fit(X_train, y_train)

y_pred = model.predict(X_test)
f1_score(y_test, y_pred, pos_label=1)

0.5441133931154181

## Bayesian Optimization and Hyperband

마지막으로는 Bayesian Optimization 방법을 활용해 초매개변수 후보들을 추출하고, Hyperband를 활용해 초매개변수 후보들을 줄여나가는 방법인 Bayesian Optimization and Hyperband를 활용해보겠다.

두 가지 방법이 있다. 하나는 optuna를 활용하는 것이고, 또 다른 하나는 HpBandSter를 활용하는 것이다. optuna를 활용하는 방법은 간단하게, sampler를 [optuna.samplers.TPESampler](https://optuna.readthedocs.io/en/stable/reference/samplers/generated/optuna.samplers.TPESampler.html)로, pruner를 [optuna.pruners.HyperbandPruner](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.pruners.HyperbandPruner.html)로 설정하는 것이다. 단, 이 방법은 실제 BOHB를 제안한 원래 논문과는 좀 차이가 있다.


In [ ]:
import optuna
from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner


study = optuna.create_study(
    # Tree Parzen Estimator를 사용한다.
    sampler=TPESampler(
        multivariate=True # 초매개변수 각각이 독립적인 확률 분포를 이루는 게 아니라,
                          # 서로 다른 종류의 초매개변수가 서로 관련 있는 확률 분포를 이룬다고 설정해야 한다.
    ),
    pruner=HyperbandPruner(reduction_factor=3), # Successive Halving을 통해 초매개변수 후보들을 줄여나간다.
    direction='maximize', # F1 Score를 최대화하는 초매개변수를 찾는다
)

optuna.logging.set_verbosity(optuna.logging.WARNING) # 불필요한 로그는 보여주지 말자.

study.optimize(
    func=objective, # 목적 함수를 의미한다
    n_trials=50, # 탐색할 초매개변수들의 개수를 의미한다
    show_progress_bar=True # 표준 출력에 프로그레스 바를 보여준다.
)

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
study.best_params

{'n_estimators': 43,
 'criterion': 'gini',
 'max_depth': 12,
 'class_weight': 'balanced_subsample',
 'min_samples_split': 0.010877274522118726}

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score


model = RandomForestClassifier(
    random_state=42,
    **study.best_params
).fit(X_train, y_train)

y_pred = model.predict(X_test)
f1_score(y_test, y_pred, pos_label=1)

0.547667937811675

실제 원본 논문과 동일하게 구현된 것은 [HpBandSter](https://github.com/automl/HpBandSter?tab=readme-ov-file)이다. 이를 써보려면 조금 복잡한 과정을 거쳐야 한다.

먼저, 아래와 같이 [hpbandster.core.worker.Worker](https://automl.github.io/HpBandSter/build/html/core/worker.html) 클래스를 구현해야 한다. 이 클래스에서 필수적으로 정의해야 할 것은  optuna 등에서 목적 함수로 활용되는 **compute**함수와 초매개변수 검색 공간을 의미하는 **get_configspace**이다.

In [ ]:
import numpy as np
import ConfigSpace as cs
from hpbandster.core.worker import Worker
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit


class RFWorker(Worker):
    def __init__(self, X, y, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.X = X
        self.y = y

    # 초매개변수 검색 공간을 정의하는 함수다.
    # 주의할 점은 staticmethod로 정의되어야 하는 것이고,
    # dict 형이 아니라, ConfigSpace라는 라이브러리에서 활용하는 자료형을 써야 한다.
    # ConfigSpace 관련 내용은 다음 API로 확인하자: https://automl.github.io/ConfigSpace/main/api/hyperparameters.html
    @staticmethod
    def get_configspace():
        space = cs.ConfigurationSpace()
        n_estimators = cs.UniformIntegerHyperparameter(name='n_estimators', lower=5, upper=200)
        criterion = cs.UniformIntegerHyperparameter(name='criterion', lower=0, upper=1) # 범주형 초매개변수는 HpBandSter가 제대로 동작하지 않는 오류가 있으니 대신 정수로 바꿔줘야 한다
        max_depth = cs.UniformIntegerHyperparameter(name='max_depth', lower=3, upper=20)
        class_weight = cs.UniformIntegerHyperparameter(name='class_weight', lower=0, upper=1) # 범주형 초매개변수는 HpBandSter가 제대로 동작하지 않는 오류가 있으니 대신 정수로 바꿔줘야 한다
        min_samples_split = cs.UniformFloatHyperparameter(name='min_samples_split', lower=0.01, upper=0.05)

        space.add_hyperparameter(n_estimators)
        space.add_hyperparameter(criterion)
        space.add_hyperparameter(max_depth)
        space.add_hyperparameter(class_weight)
        space.add_hyperparameter(min_samples_split)

        return space

    # optuna의 목적 함수와 동일한 역할을 한다.
    def compute(self,
                config, # 초매개변수 후보가 있는 dict 자료형이다,
                budget, # Successive Halving 및 Hyperband에서 사용하는 자원을 의미한다.
                **kwargs
        ):
        # 추출된 초매개변수를 가져온다.
        n_estimators = config['n_estimators']
        criterion = ['gini', 'entropy'][config['criterion']]
        max_depth = config['max_depth']
        class_weight = ['balanced', 'balanced_subsample'][config['class_weight']]
        min_samples_split = config['min_samples_split']

        # 자원 양에 따라서 사용할 데이터의 양을 달리한다.
        if budget < 1.0:
            splitter = StratifiedShuffleSplit(n_splits=1, train_size=budget, random_state=42)
            I_resource, _ = next(splitter.split(self.X, self.y))
            X_resource, y_resource = self.X[I_resource], self.y[I_resource]
        else:
            X_resource, y_resource = self.X, self.y

        # 다음은 계속 했던 학습 모델 훈련 및 성능 추정이다.
        splitter = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
        scores = []

        for I_train, I_test in splitter.split(X_resource, y_resource):
            X_inner_train, X_inner_test, y_inner_train, y_inner_test = X_resource[I_train], X_resource[I_test], y_resource[I_train], y_resource[I_test]
            model = RandomForestClassifier(
                random_state=42,
                n_estimators=n_estimators,
                criterion=criterion,
                max_depth=max_depth,
                class_weight=class_weight,
                min_samples_split=min_samples_split
            )
            model.fit(X_inner_train, y_inner_train)
            y_pred = model.predict(X_inner_test)
            f1 = f1_score(y_true=y_inner_test, y_pred=y_pred, pos_label=1)
            scores.append(f1)

        # 마지막으로, 출력값은 무조건 loss와 info라는 key를 가진 dict 자료형이어야 한다.
        # 주의할점은 loss 값이 음수가 되면 제대로 학습이 되지 않는다.
        # 또한, loss에서 알 수 있듯이 HpBandSter는 성능치를 최소화하는 방향으로만 작동한다.
        # 따라서, 1에서 F1 Score을 빼주는 방식으로 loss를 구성했다.
        return {
            'loss': 1 - np.mean(scores),
            'info': config
        }

이렇게 Worker를 구현했다면, 별도의 [hpbandster.core.nameserver.NameServer](https://automl.github.io/HpBandSter/build/html/core/nameserver.html)를 이용해 별도의 서버를 실행하고, 그 서버 내에서 위에 구현한 Worker를 실행해야 한다.

In [ ]:
from hpbandster.core.nameserver import NameServer


RUN_ID = 'opt-rf-w-bohb' # 작업 ID를 의미한다.
HOST = 'localhost' # 서버의 주소
PORT = 9090 # 서버의 포트


# 아래와 같이 서버를 실행한다.
name_server = NameServer(run_id=RUN_ID, host=HOST, port=PORT)
name_server.start()

# 위에서 구현한 Worker를 위에 설정한 서버 상에서 실행한다.
worker = RFWorker(X=X_train, y=y_train, run_id=RUN_ID, nameserver=HOST, nameserver_port=PORT)
worker.run(background=True)

마지막으로는, [hpbandster.optimizers.BOHB](https://automl.github.io/HpBandSter/build/html/optimizers/bohb.html)를 실행해서 최적의 매개변수를 찾는 것이다.


In [ ]:
from hpbandster.optimizers import BOHB


bohb = BOHB(
    configspace = RFWorker.get_configspace(), # 검색 공간을 정의한다.
    run_id=RUN_ID,
    nameserver=HOST,
    nameserver_port=PORT,
    min_budget=0.01, # 최소 자원양을 의미한다.
    max_budget=1.0 # 최대 자원양을 의미한다.
)
result = bohb.run(n_iterations=50)

다 끝났다면 BOHB와 서버를 종료한다.

In [ ]:
bohb.shutdown(shutdown_workers=True)
name_server.shutdown()

가장 최적의 초매개변수를 찾기 위해서는 일단 최적의 작업 ID를 가져와야 한다. 다음을 확인해보자.

In [ ]:
best_trial_id = result.get_incumbent_id()
best_trial_id

(48, 0, 1)

초매개변수들 중 이러한 ID를 가진 것을 가져오면 된다. 전체 초매개변수들은 다음에 저장되어 있다.

In [ ]:
hyperparameters = result.get_id2config_mapping()
hyperparameters

{(0,
  0,
  0): {'config': {'class_weight': 0,
   'criterion': 1,
   'max_depth': 17,
   'min_samples_split': 0.027970181241246443,
   'n_estimators': 127}, 'config_info': {'model_based_pick': False}},
 (0,
  0,
  1): {'config': {'class_weight': 0,
   'criterion': 0,
   'max_depth': 20,
   'min_samples_split': 0.027054761994011828,
   'n_estimators': 106}, 'config_info': {'model_based_pick': False}},
 (0,
  0,
  2): {'config': {'class_weight': 0,
   'criterion': 1,
   'max_depth': 20,
   'min_samples_split': 0.010412808702363163,
   'n_estimators': 68}, 'config_info': {'model_based_pick': False}},
 (0,
  0,
  3): {'config': {'class_weight': 1,
   'criterion': 0,
   'max_depth': 19,
   'min_samples_split': 0.015801038820204907,
   'n_estimators': 38}, 'config_info': {'model_based_pick': False}},
 (0,
  0,
  4): {'config': {'class_weight': 0,
   'criterion': 0,
   'max_depth': 7,
   'min_samples_split': 0.03980010757298404,
   'n_estimators': 167}, 'config_info': {'model_based_pick': Fal

이제 최적의 ID에 해당하는 설정을 출력해보자.

In [ ]:
hyperparameters[best_trial_id]

{'config': {'class_weight': 1,
  'criterion': 1,
  'max_depth': 20,
  'min_samples_split': 0.010252955151271581,
  'n_estimators': 174},
 'config_info': {'model_based_pick': True}}

In [ ]:
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier


model = RandomForestClassifier(
    random_state=42,
    class_weight='balanced_subsample',
    criterion='entropy',
    max_depth=20,
    min_samples_split=0.010252955151271581,
    n_estimators=174
).fit(X_train, y_train)

y_pred = model.predict(X_test)
f1_score(y_test, y_pred, pos_label=1)

0.5489398780133605